# Authors - Justin Clifton

---
## Table of Contents (Work In Progress)

1. [Introduction](#intro)
---

## Introduction <a class="anchor" id="intro"></a>

In this notebook, we will be extending the work that was done in this [notebook](https://github.com/juliajanu/Math475_Project_4-/blob/master/nba_stats.ipynb) by using the dataset that was created. We will analyze the data and attempt to create a useful model for predicting the NBA MVP.

In [6]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split

#models
from sklearn.ensemble import RandomForestClassifier as RFC
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.ensemble import StackingClassifier

#feature selection
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV

#scalers
from sklearn.preprocessing import StandardScaler

#score function
from sklearn.metrics import f1_score

In [7]:
stats_df = pd.read_csv('NBA_Stats_MVP.csv')

In [8]:
stats_df.head()

,Year,Pos,Age,Tm,G,GS,MP,PER,TS%,3PAr,...,TRB,AST,STL,BLK,TOV,PF,PTS,is_allstar,Name,was_mvp
0,1974.0,C,27.0,HOU,79.0,NaN,2459.0,15.9,0.516,NaN,...,923.0,166.0,80.0,104.0,NaN,227.0,865.0,0,Zaid Abdul-Aziz,0
1,1974.0,C,26.0,MIL,81.0,NaN,3548.0,24.4,0.564,NaN,...,1178.0,386.0,112.0,283.0,NaN,238.0,2191.0,1,Kareem Abdul-Jabbar,0
2,1974.0,SF,26.0,DET,74.0,NaN,2298.0,10.9,0.457,NaN,...,448.0,141.0,110.0,12.0,NaN,242.0,759.0,0,Don Adams,0
3,1974.0,PG,27.0,CHI,55.0,NaN,618.0,10.0,0.447,NaN,...,69.0,56.0,36.0,1.0,NaN,63.0,182.0,0,Rick Adelman,0
4,1974.0,PG,26.0,MIL,72.0,NaN,2388.0,18.8,0.536,NaN,...,291.0,374.0,137.0,22.0,NaN,215.0,1268.0,0,Lucius Allen,0


In [9]:
stats_df.describe()

,Year,Age,G,GS,MP,PER,TS%,3PAr,FTr,ORB%,...,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,is_allstar,was_mvp
count,20797.000000,20797.000000,20797.000000,18233.000000,20797.000000,20792.000000,20715.000000,18839.000000,20703.000000,20792.000000,...,20797.000000,20797.000000,20797.000000,20797.000000,20797.000000,19645.000000,20797.000000,20797.000000,20797.000000,20797.000000
mean,1998.155840,26.773477,50.263836,23.593375,1180.121796,12.453867,0.502274,0.158604,0.318799,6.181565,...,147.199404,209.388614,112.975237,39.897052,24.470260,73.939832,111.226860,496.276242,0.039813,0.002116
std,12.232484,3.945470,26.681105,28.632387,929.215744,6.100548,0.093337,0.187495,0.226552,4.872685,...,145.921912,207.772110,137.240043,38.713053,36.935084,67.713803,82.445297,484.090240,0.195525,0.045949
min,1974.000000,18.000000,1.000000,0.000000,0.000000,-90.600000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1988.000000,24.000000,27.000000,0.000000,325.000000,9.800000,0.471000,0.005000,0.199000,2.600000,...,33.000000,48.000000,18.000000,9.000000,3.000000,18.000000,37.000000,100.000000,0.000000,0.000000
50%,1999.000000,26.000000,56.000000,8.000000,1013.000000,12.700000,0.514000,0.064000,0.286000,5.400000,...,106.000000,149.000000,64.000000,29.000000,11.000000,55.000000,102.000000,348.000000,0.000000,0.000000
75%,2009.000000,29.000000,76.000000,45.000000,1926.000000,15.600000,0.549000,0.288000,0.392000,9.000000,...,212.000000,302.000000,157.000000,60.000000,29.000000,112.000000,174.000000,764.000000,0.000000,0.000000
max,2017.000000,44.000000,87.000000,83.000000,3638.000000,129.100000,1.136000,1.000000,6.000000,100.000000,...,1111.000000,1530.000000,1164.000000,301.000000,456.000000,464.000000,386.000000,3041.000000,1.000000,1.000000


In [10]:
stats_df.isnull().sum() > 0

Year          False
Pos           False
Age           False
Tm            False
G             False
GS             True
MP            False
PER            True
TS%            True
3PAr           True
FTr            True
ORB%           True
DRB%           True
TRB%           True
AST%           True
STL%           True
BLK%           True
TOV%           True
USG%           True
OWS           False
DWS           False
WS            False
WS/48          True
OBPM          False
DBPM          False
BPM           False
VORP          False
FG            False
FGA           False
FG%            True
3P             True
3PA            True
3P%            True
2P            False
2PA           False
2P%            True
eFG%           True
FT            False
FTA           False
FT%            True
ORB           False
DRB           False
TRB           False
AST           False
STL           False
BLK           False
TOV            True
PF            False
PTS           False
is_allstar    False
